# TablePC-Prize-Classification-Data-Analysis

Analizimiz için gerekli olan kütüphanelerimizi çekirdeğe dahil edelim.

In [ ]:


import numpy as np
import seaborn as sns
import pandas as pd
import missingno           

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plot 
import os
from subprocess import check_output
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from warnings import filterwarnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix as cm
from sklearn import ensemble
filterwarnings('ignore')

sns.set(rc={'figure.figsize':(10,8)})



# VERİYİ İŞLEME VE KONTROL ETME (Load and Check Data)

Veriler üzerinde işlem ve analiz yapabilmek için onu bir dataframe olarak atayıp df değişkenine kopyalayarak kullanalım ve gösterelim.

In [ ]:
df = pd.read_csv('../input/tabletpc-priceclassification/tablet.csv')


In [ ]:


df  

Veri setinin kaç öznitelikten oluştuğunu görüntüleyelim

In [ ]:
df.shape #2000 x 20 lik bir matrisimiz var. 2000 satır ve 20 sütundan oluşuyor.  20 bağımlı ya da bağımsız öznitelikte 2000 tablet cihazı bulunuyor.

Veri setinin kaç gözlemden oluştuğunu görüntüleyelim.

In [ ]:
df.count() #2000 tane gözlemden 5 tanesinin OnKameraMP, 12 tanesinin RAM öznitelik bilgisi yok.

Veri setinin özniteliklerini inceleyelim.

In [ ]:
df.columns 

Veri setinin ilk 5 gözlemini yazdıralım.

In [ ]:
df.head()

# Değişken Görüntüleme (Variable Description)

    1. BataryaGucu: Ürünlerin b sayısal olarak güçleri.
    2. Blutooth: Var(1) ya da yok(0)
    3. MicroislemciHizi: Ürünlerin  sayısal olarak mikroşlemci hızları
    4. CiftHat: Var(1) ya da yok(0)
    5. OnKameraMp: Ürünlerin ön kamera çözünürlükleri
    6. 4G: Var(1) ya da yok(0)
    7. DahiliBellek: Ürünlerin sayısal olarak dahili bellekleri
    8. Kalinlik: Ürünlerin sayısal olarak kalınlıkları
    9. Agirlik: Ürünlerin sayısal olarak ağırlıkları
    10. CekirdekSayisi: Ürünlerin  çekirdek sayıları
    11. ArkaKameraMP: Ürünlerin arka kamera çözünürlüğü
    12. CozunurlukYükseklik: Ürünlerin çözünürlük yüksekliği
    13. CozunurlukGenislik: Ürünlerin çözünürlük genişliği
    14. RAM: Ürünlerin rastgele değişimli hafızaları
    15. BataryaOmru: Ürünlerin batarya ömürleri
    16. 3G: Var(1) ya da yok(0)
    17. Dokunmatik : Var(1) ya da yok(0)
    18. WiFi: Var(1) ya da yok(0)
    19. FiyatAraligi: Fiyat aralığının nesnesel olarak 3 gözlemi (ucuz,pahalı,normal)
    20. Renk: Ürünlerin nesnesel olarak renkleri
     
    
    

In [ ]:
df.dtypes #Bu 20 değişkenin veri tipi






Veri setindeki değişkenlerin hangi tipte olduklarını inceleyelim.

In [ ]:

df.info() #Gözlemlerin öznitelikleri genel olarak sayısal tipte. 3 tane veri tipi(int,object,float) bulunuyor. Hiçbiri için null değeri atanmamış.

# Hedef Değişkenin ve Özniteliklerin Dağılımı

Hedef değişken burada tabletin ana ve diğer özelliklerine bağlı olarak değişen fiyat aralığıdır. Örneğin pahalı bir tabletin ram kamera çözünürlük değerleri yüksek olurken ucuz bir tabletin bu özellikleri daha düşük  olabilir. Bu veri seti bize tablet cihazları verilerinin fiyat analizini yapmayı hedefler. Bir satış mağazasında vitrine koymak için hangi ürünleri koymalıyız? Koyacağımız ürün ucuz, iyi özellikli ve ortalama olarak herkese hitap eden bir ürün olmalıdır. Bu da tabletin özniteliklerine göre değişir durumdadır.

In [ ]:
df["FiyatAraligi"].nunique()

In [ ]:
df["FiyatAraligi"].unique() #4 tane fiyat aralığı tipimiz var.

In [ ]:
df.FiyatAraligi.value_counts() #Her bir fiyat değerinden toplam 500 gözlem var.

In [ ]:
sns.countplot(x = "FiyatAraligi", data = df);


Genel bir histogram çizdirelim

In [ ]:
df.hist()#Genel olarak bakıldığında çoğu öznitelik için gözlemler yakın şekilde dağılmış.

In [ ]:
sns.scatterplot(x = "ArkaKameraMP", y = "OnKameraMP", hue = "FiyatAraligi", data = df); #Örneğin ön kamera ve arka kamera arasındaki fiyat aralığı dağılımına bakalım. Değerler üçgen şeklinde artarak seyrediyor. Arka kamera çözünürlüğü arttıkça ön kamera çözünürlüğü de artıyor. Doğru orantısız değerler de var.


In [ ]:
sns.scatterplot(x = "CozunurlukGenislik", y = "CozunurlukYükseklik", hue = "FiyatAraligi", data = df); #Çözünürlük genişlik ve yükseklik arasında da genel olarak doğrusal bir ilişki var.

In [ ]:
sns.scatterplot(x = "BataryaGucu", y = "BataryaOmru", hue = "FiyatAraligi", data = df); #Batarya gücünün batarya ömrüyle özel bir ilgisi yok. Hepsi değişken değerdeler.

In [ ]:
sns.violinplot(x = "RAM", data = df); #Ram değeri fiyatı etkileyen çok önemli bir özellik. Dağılım olarak da düzgüne yakın bir şekilde.

In [ ]:
sns.distplot(df["RAM"], bins=32, color="red"); #Bu grafikten de adet olarak RAM için  her değerden birbirine yakın dağılım olduğu net anlaşılıyor.


RAM dağılımını fiyat aralığı üzerinde görelim.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "RAM", data = df); #Görüldüğü gibi Ram değeri yüksek olan değerlerin fiyat aralığı buna bağlı olarak değişiyor. Ram değeri ne kadar yüksek olursa fiyat da  artıyor. Buradan genel bir ilişki kurabiliriz.


In [ ]:
sns.jointplot(x = df["OnKameraMP"], y = df["ArkaKameraMP"], kind = "kde", color = "brown")


In [ ]:
plot.hist(df['RAM'], bins=15)#Ram ile ilgili histogram çizdirdiğimizde;

In [ ]:
sns.relplot(x='OnKameraMP', y='ArkaKameraMP', hue='FiyatAraligi', size='FiyatAraligi', col='FiyatAraligi', data=df)

# Eksik Verilerin Tespiti ve Doldurma

In [ ]:
df.isna().sum() #OnKameraMPde 5 RAMde 12 eksik gözlem var.

Veri çerçevesinde kaç eksik gözlem olduğunu görelim.

In [ ]:
df.isnull().sum().sum() #Toplam kaç eksik değer olduğunu gözlemliyoruz.

Eksik değerleri barındıran 2 özniteliğimiz var. Acaba bu eksik verilerin birbirleriyle bir ilişkisi var mı yoksa rastgele mi bunu gözlemlemek için missingno kütüphanesinin görselleştirmesiden yararlanalım.

In [ ]:
missingno.matrix(df,figsize=(20, 10)); 

Her bir beyaz çizgi eksik veriyi temsil ediyor. Burdan görülen eksik değerlerden bir çıkarımda bulunabiliriz fakat eksik verilerin tamamı gözükmüyor.Bunu daha iyi irdelyebilmek için ısı haritası çizdirelim.

In [ ]:
missingno.heatmap(df, figsize= (20,10));

Ön kamera çözünürlüğü ve Ram değerlerinin aynı gözlemde eksik veri bulundurmadığını aralarında sayısal bir ilişki olmadığından ve bir ısı yoğunluğu olmadığından anlıyoruz. Aralarında hiçbir ilişki yok. Yani eksik veri barındırmalarının sebebinde birbirleriyle bir ilişki yok. 

Bu eksik gözlemler toplam gözlemlerin yüzde kaçını barındırıyor onu hesaplayalım

In [ ]:
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son

In [ ]:
eksik_deger_tablosu(df) #Görüldüğü üzere eksik veriler gözlemlerin yüzde olarak çok küçük bir kısmını oluşturuyorlar ve aralarında bir ilişki yok. Yüzde olarak 1'in altındalar. Yani bir çıkarımda bulunmak için ciddiye alınacak bir yüzdede değiller. Fakat eksik olmalarının diğer bağımlı öznitelikler ile alakalı mı değil mi bunu irdeleyebiriz.

Bu eksik verilerimizi doldurma işlemine geçelim.

In [ ]:
df.groupby("FiyatAraligi").mean() # Fiyat aralığına göre eksik ram ve on kamera değerlerini en uygun şekilde doldurmak için groupbyı kullanalım.

Tüm özniteliklere gerek yok. Sadece eksik olan öznitelikleri inceleyelim.

In [ ]:
df.groupby("FiyatAraligi")[["OnKameraMP"]].mean() 

Ön kamera çözünürlüğü eksik olan fiyat aralıklarına bakalım.

Peki bu kamera çözürlüğünün eksik verilerinin fiyat aralığı bilgisiyle bir alakası olabilir mi?

In [ ]:
sns.countplot(df[df["OnKameraMP"].isnull()]["FiyatAraligi"]);  #Grafikte görüldüğü gibi bir alakası var.

In [ ]:
df[(df["FiyatAraligi"] == "Ucuz") & (df["OnKameraMP"].isnull())]
df[(df["FiyatAraligi"] == "Normal") & (df["OnKameraMP"].isnull())]
df[(df["FiyatAraligi"] == "Pahalı") & (df["OnKameraMP"].isnull())]
df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())] # Görüldüğü üzere sadece çok ucuz olan fiyat aralığındaki ön kamera bilgileri nan değerinde. Yani eksik verinin olmasının sebebi fiyat aralığının çok ucuz olmasıyla alakalı.

In [ ]:
cok_ucuz_OnKameraMP = df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())].index
cok_ucuz_OnKameraMP

In [ ]:
df.loc[cok_ucuz_OnKameraMP,"OnKameraMP"] = 4 # Eksik değerleri 4 ile dolduralım. Çünkü fiyat aralıklarına göre çok ucuz ortalama değeri 4 değerine yakın

Kontrol edelim

In [ ]:
df.isna().sum()["OnKameraMP"] # Görüldüğü üzere artık hiçbir ön kamera çözünürlüğünde eksik değer yok

Aynı işlemi RAM için uygulayalım. Acaba RAM deki eksik verilerin fiyat aralığıyla bir ilgisi var mı?

In [ ]:
df.groupby("FiyatAraligi")[["RAM"]].mean() 

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["FiyatAraligi"]); #Görüldüğü üzere pahalı fiyat aralığındaki cihazların RAM bilgisi eksik. Burdan bir çıkarım yapabiliriz.

In [ ]:

df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())] #Toplam 12 eksik değer de pahalı olan fiyat aralığında


In [ ]:
pahali_RAM = df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())].index
pahali_RAM

In [ ]:
df.loc[pahali_RAM,"RAM"] = 3450 # Pahalı fiyat aralığındaki eksik RAM bilgilerini 3450 yaptık çünkü ortalama olarak 3450. 

In [ ]:
df.isna().sum()["RAM"] #Şimdi kontrol edelim. Görüldüğü üzere artık hiçbir eksik değer kalmadı.

Böylece eksik verilerin fiyat aralığıyla bir ilişkisi olduğunu bulmuş olduk ve eksik verileri ortalama olarak fiyat aralığına göre uygun bir şekilde doldurmuş olduk.

# KATEGORİK DÖNÜŞÜM

Test,train,split uygulamalarını bağımlı,bağımsız değişkenleri belirlemek için bazı string verileri sayısallaştırmamız gerekir. Bunun için istenilen özniteliği preprocessing kütüphanesinin verilerinden yararlanarak bir dönüşüm yaparız.

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
df["Bluetooth"] = le.fit_transform(df["Bluetooth"])
df.head()
#Yok =1, var =0.Burda kütüphane değerlerin numaralandırmasını kendi seçtiği için doğruluk olarak vara 1 yoka 0 veremiyoruz. Zaten yaptığımız işlemlerde onun doğruluğuna göre değil numaralandırmış olarak var yok olduğunu tutuyoruz. Fakat bunun için dummies formüllerini kullanabiliriz.


In [ ]:
df["CiftHat"] = le.fit_transform(df["CiftHat"])
df.head() #Yok =1, var =0

In [ ]:

df["4G"] = le.fit_transform(df["4G"])
df.head() #Yok =1, var =0

In [ ]:
df["Dokunmatik"] = le.fit_transform(df["Dokunmatik"])
df.head() #Yok =1, var =0

In [ ]:
df["3G"] = le.fit_transform(df["3G"])
df.head()

In [ ]:
df["WiFi"] = le.fit_transform(df["WiFi"])
df.head()

In [ ]:
df["Renk_Encoded"] = le.fit_transform(df["Renk"])
df.head() #Renk değeri çok fazla tip içerdiği için onu encoded olarak neye eşit olduğunu anlamak için farklı bir öznitelik tanımladık.

In [ ]:
df["FiyatAraligi_Encoded"] =  le.fit_transform(df["FiyatAraligi"])
df.head()

In [ ]:
df["Renk"].unique()

In [ ]:
df["Renk"].nunique() #12 tane benzersiz renk değerimiz var

In [ ]:


df.describe().T

In [ ]:
df.groupby(["FiyatAraligi"]).mean()  #Fiyat aralığına göre ortalama değerlerine baktığımızda pahalı değerlerin çoğu öznitelik için daha fazla olduğunu görebiliriz.


In [ ]:
df.groupby(["FiyatAraligi"]).std() #Standart sapma değerlerine baktığımızda da genel olarak  fiyat aralıklarının değerleri birbirine yakın değerde. Herhangi bir önçıkarımda bulunmak zor.

In [ ]:
df.groupby(["FiyatAraligi"]).max() #Max değerlerine baktığımızda ise  belirleyici öznitelikler dışında değerler birbirine eşit. Veri seti maksimum alabilecekleri değerde.

# Korelasyon Matrisi

Korelasyon matrisini çizdirelim.

In [ ]:
df.corr() #Korelasyon matrisi 20x20 olmak üzere genel olarak ilişkilendirmelere bakıldığında çoğunun negatif yönde bir ilişkisi olduğu görülüyor. 1'e yakın pozitif yönde ilişkiler çok az.

En güçlü pozitif ilişki hangi değişkenler arasında gösterelim.

In [ ]:
df.corr()["ArkaKameraMP"]["OnKameraMP"] #En güçlü pozitif ilişki 






Korelasyon matrisini daha iyi okuyabilmek için ısı haritası çizdirelim.

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values) #Genel olarak ısı haritasına baktığımızda negatife doğru ilişkilerin çoğunluğunu görüyoruz. Her bir öznitelik arasında neredeyse negatif bir ilişki var.

Yalnızca sayısal sütunları ve 1den çok benzersiz sütunları  saklayarak bir çekirdek yoğunluğu grafiği çizdirelim. Ters çevrilmesi için de sütun sayısını azaltalım.

In [ ]:

def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) 
 
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] 
    columnNames = list(df)
    if len(columnNames) > 10: 
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plot.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plot.suptitle('Scatter and Density Plot')
    plot.show()
    

In [ ]:
plotScatterMatrix(df,20,10) #Sadece sayısal özniteliklerden ve birden fazla benzersiz değerli özniteliklerden oluşan çekirdek yoğunluğu grafiğimiz. Yoğunluğun bazı yerlerde çok bazı yerlerde az olduğunu gözlemliyoruz. Bu korelasyon matrisine bağlı olarak gelişen pozitif ve negatif ilişkileri gösteren yoğunluk

# Bağımlı ve Bağımsız Değişkenler

Bağımlı ve bağımsız değişkenleri belirleyip regresyon tahmini yapmak için modelimizi oluşturalım.


In [ ]:
linear_regresyon = LinearRegression() 


In [ ]:
X = df.drop(["FiyatAraligi_Encoded","FiyatAraligi","Renk"], axis = 1) #Renk özniteliğini düşürdük çünkü renk object değer içerdiği için onu float türüne dönüştüremiyorduk. O yüzden numeration yaptığımız encodedı kullandık. 

y = df["FiyatAraligi_Encoded"] 

In [ ]:
X# Bağımlı değişkenimizi etkileyen öznitelikler. Bağımsız değişkenler

In [ ]:
y #Bağımlı değişkenimiz. Fiyat aralığımız cihazın diğer özelliklerine bağlı olarak değişiyor.


Denklemimizi oluşturalım

In [ ]:
linear_regresyon.fit(X, y)



Modelimize rastgele değerlerle tahmin ettirelim.

In [ ]:
linear_regresyon.predict([[1233,1,1,1,2,0,50,0.1,146,1,10,499,695,2000,2,0,1,1,0,]]) 

In [ ]:
df["model1_tahmin"] = linear_regresyon.predict(X)#Oluşturulan tahmin modeli yaklaşık olarak gerçek değere yakınsıyor.
df

# Değişkenlerin Bölütlenmesi

Bağımlı ve bağımsız değişkenler uygun şekildeyse onları eğitim ve test olarak bölütleyelim.

In [ ]:
X 


In [ ]:
y

Test oranı yüzde 25 olmak üzere bölütleme işlemini gerçekleştirelim.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

Bölütleme işlemini eğitim ve test verilerine bakarak kontrol edip inceleyelim.

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

# GaussianNB Modeli ve Eğitimi

Şimdi bağımlı ve bağımsız olarak ayırıp eğittimiz değişkenleri GaussianNB Modeli oluşturarak işlemler yapalım

In [ ]:
NB = GaussianNB()
NB_model = NB.fit(X_train,y_train)

Modelin alabileceği parametreleri görüntüleyelim

In [ ]:
NB_model

Modelin üzerinde yazılabilecek tüm komutlar

In [ ]:


dir(NB_model)

Modelin eğitimde kullanılmayan x testin ilk 20 gözleminin fiyat aralığını tahmin ettirmeye çalışalım.

In [ ]:
X_test[0:20]

Modelin tahmin ettiği X_test gözlemlerinin baştan itibaren 20 tanesini inceleyelim.

In [ ]:
NB_model.predict(X_test)[0:20]

Modelden farklı olarak gözlemlerin  gerçek verilerine bakalım.

In [ ]:
y_test[0:20]

Görüldüğü üzere bazılarını model doğru tahmin etse de farklı değerler var.


y_pred adlı bir değişken oluşturup X_test tahmin sonuclarını bu değişkene atayalım.

In [ ]:
y_pred = NB_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

Accuracy skoru yani doğruluk skorunu görüntüleyim. Bundan kastımız ise tahmin ve gerçek değerler karşılaştırıldığında ne kadar doğru  tahminlerimiz var oranlamak. Amaç bunu görmek.

In [ ]:
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini(confusion matrix) görüntülüyelim. 

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
karmasiklik_matrisi #4 x 4lük bir karmaşıklık matrisimiz var.

Karmaşık matris skorunu yani accuracy score'u hesaplayalım. Burdaki amaç ise doğruluk skorunu matematiksel olarak hesaplamak.

In [ ]:
(karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1]) / (karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1] +  karmasiklik_matrisi[1][0] + karmasiklik_matrisi[0][1])

Görüldüğü gibi matemiksel olarak hesapladığımızda tahmin karşılaştırmasından hesaplanan doğruluk skorundan farklı olarak 0.06 gibi bir hata payı ortaya çıkıyor.

Modeli farklı gözlemlerle doldurup tekrardan skorunu hesaplayalım. Yani cross validation scoreunu,katlamalı çapraz doğrulamasını yapalım.

In [ ]:
cross_val_score(NB_model, X_test, y_test, cv = 20)

In [ ]:
cross_val_score(NB_model, X_test, y_test, cv = 20).mean() #Ortalamalarını alalım.

Classification report ile  precision reccal f1-score değerlerini yazdıralım. Yani sınıflandırma raporunu çizelim.

In [ ]:
print(classification_report(y_test, y_pred))#Tablo 0.71 ile 0.93 arasında yaklaşık olarak değişkenlik gösteriyor.

Bu değerlere daha sonrasında  analiz etmek ve net görmek için teker teker görüntüleyelim.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore #Kesinlik skoru

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore #Yakamala, hassaslık skoru

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score #F1 skoru

# Decision Tree Ve Eğitimi

Karar ağacı modelimizi inşa edelim ve eğitelim.

In [ ]:
#cart = DecisionTreeClassifier(random_state = 42)
cart = DecisionTreeClassifier(random_state = 42, criterion='gini')
cart_model_gini = cart.fit(X_train, y_train)

In [ ]:
cart_model_gini #Karar ağacı kütüphanesinin parametreleri

Karar ağacı modeli için karmaşıklık matrisini ve accuracy değerini hesaplayalım.


In [ ]:
y_pred_gini = cart_model_gini.predict(X_test)


In [ ]:
accuracy_score(y_test, y_pred_gini)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred_gini)
print(karmasiklik_matrisi)#Değerler sayısal olarak birbiriyle çok dengesiz.

In [ ]:
cross_val_score(cart_model_gini, X, y, cv = 20)# 20 tane değer için baktığımızda sayılar birbirine çok yakın mutlak olarak sayıların arasındaki fark birbirine yakın değerlerde

In [ ]:
cross_val_score(cart_model_gini, X, y, cv = 20).mean()#Ortalamaları

In [ ]:
print(classification_report(y_test, y_pred_gini))#Genel olarak skor değerlerine baktığımızda 0.10-0.90 arasında değişkenlik gösteriyor. 3 için neredeyse skor değerleri aynı.

Şimdi girdiğimiz gini criterion değerini entropy olarak değiştirip score değerlerini buna göre görüntüleyelim.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

In [ ]:
cart_model #Information Gain hesaplamak için criterionu entropy olarak girdik.


In [ ]:
df.columns

In [ ]:
y_pred2 = cart_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred2)#Accuracy score değeri criterion  gini iken 0.816 değerini göstermişti. Fakat entropy için 0.852 çıktı. Yaklaşık 0.036lık bir artış var. Bu değer daha başarılı bir sonuç veriyor.

In [ ]:


karmasiklik_matrisi = confusion_matrix(y_test, y_pred2)
print(karmasiklik_matrisi)#Veriler birbirleriyle dengesiz. Fakat simetrik olarak baktığımızda birbirlerine yakın.

In [ ]:
cross_val_score(cart_model, X, y, cv = 20)

In [ ]:
cross_val_score(cart_model, X, y, cv = 20).mean()#Gini'deki ortalama 0.8440000000000001'di. Aralarında yaklaşık olarak 0.006lik bir fark var.

In [ ]:
print(classification_report(y_test, y_pred2)) #Skor değerleri 0.79-0.90 arasında değişken. 


Modelin karar ağacını çizdirelim.

In [ ]:
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
graph = Source(tree.export_graphviz(cart, out_file = None, feature_names = X.columns, filled = True))
display(SVG(graph.pipe(format = 'svg'))) #Görüldüğü üzere karar ağacının kök düğümüne baktığımızda RAM özniteliğine dayanarak karar verdiğini görüyoruz.

Karar ağacına göre özniteliklerin önem derecesini görüntüleyelim.

In [ ]:
ranking = cart.feature_importances_
features = np.argsort(ranking)[::-1][:20]
columns = X.columns

plot.figure(figsize = (16, 9))
plot.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plot.bar(range(len(features)), ranking[features], color="lime", align="center")
plot.xticks(range(len(features)), columns[features], rotation=80)
plot.show()#Tabloya baktığımızda anlatılmak istenen aslında fiyat aralığının en çok ram değerine göre değiştiğini gösterir. Fiyat biçilmek istenen yeni bir cihazda en çok ram değerine bakılarak bir fiyat biçilebilir.


# KNN Modeli ve Eğitimi

KNN modelini oluşturup skor değerlerini normal gözlemlerle görüntüleyelim.

In [ ]:

knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

In [ ]:
knn_model

In [ ]:
y_pred3 = knn_model.predict(X)

In [ ]:
accuracy_score(y, y_pred3)#Tüm öznitelikler için doğruluk skorunu görüntüledik. SAdece test için yapmadık.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y, y_pred3)
print(karmasiklik_matrisi) #Matrisin köşegeni 

In [ ]:
cross_val_score(knn_model, X_test, y_test, cv = 20)

In [ ]:
cross_val_score(knn_model, X_test, y_test, cv = 20).mean()


In [ ]:
print(classification_report(y, y_pred3))#Değerler diğer eğitimlerin raporlarına göre daha yüksek oranda. Bu modelin daha başarılı olduğunu söyleyebiliriz.



KNN modelini parametrelerin optimizasyonu yaparak tanımlayalım ve eğitelim.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)}
knn_params

En yakın komşu sayısını 2 ile 15 arasında hepsini deneyecek.

In [ ]:
knn_komsu = KNeighborsClassifier()
knn_cv = GridSearchCV(knn_komsu, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(9)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test) 

In [ ]:
accuracy_score(y_test, y_pred) #Test ve ayarlanmış değerler için  doğruluk skoru. En iyi skor ile aynı değere sahip.

Modelin k değerini nasıl seçtiğini görüntülemek için 2 ile 15 arası  k değeri için doğruluk skorlarını görselleştirelim.

In [ ]:
skor_listesi = []

for each in range(2,15):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    skor_listesi.append(knn2.score(X_test, y_test))
    
plot.plot(range(2,15),skor_listesi)
plot.xlabel("k değerleri")
plot.ylabel("doğruluk skoru")
plot.show()    #Biz en iyi skor değerini arrayde 9 için bulmuştuk. Grafikte de 9'un denk geldiği yer sonucunu bulduğumuz gibi 0.930 ve 0.935 arasında. Yani bizim asıl noktamız 9'a konum olarak yakın bir yerde. En iyi komşu olarak seçmesi ve knn modeline göre algoritmik olarak nokta 9'a yakındır.

  Model için  en iyi komşu olarak ayarlanmış gözlem ile skorunu tekrar hesaplayalım.

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 20)

In [ ]:


cross_val_score(knn_tuned, X_test, y_test, cv = 20).mean() #Önceki normal cross val score u normal gözlemler için 0.8870898984268549 iken komşu sayılarıyla birlikte tekrar gözlemlediğimizde 0.9047537160906728 çıktı. Denediğimiz komşu sayıların gözlemleri için daha yüksek.
 

Genel olarak 3 eğitimin de başarı karşılaştırmasını yaptığımızda KNN modeli için sınıflandırma raporunun daha başarılı bir oranda olduğunu söyleyebiliriz. Doğruluk skor karşılaştırması yaptığımızda da en yüksek skorun KNN modeline ait olduğunu görebiliriz. Yani bu veri seti için KNN modeli ile analiz yapmak daha doğru olacaktır. Yapılan analiz daha kapsamlıdır.